In [18]:
"""
Used for ploting csv data
"""

'\nUsed for ploting csv data\n'

In [19]:
# imports
import time
import pandas as pd
import base_strategy as bs
import init_data_helper as idh
from fractions import Fraction as frac
from tests.test_all_tests import get_test_data_path

from binance import AsyncClient, Client
import api_data as api # for Binance data
from Historic_Crypto import HistoricalData # for CoinBase Pro

In [20]:
# Read in csv's
list_of_csv = [
    # TODO: figure out problems with 2017 data
    # 'full_data__6__2017.csv', # don't use 2017 data, its not great
    # 'full_data__6__2018.csv',
    # 'full_data__6__2019.csv',
    # 'full_data__6__2020.csv',
    # 'full_data__6__2021.csv'
]
sorted_list_of_csv = [
    # 'sorted_full_data_2017.csv',
    'sorted_full_data_2018.csv',
    'sorted_full_data_2019.csv',
    'sorted_full_data_2020.csv',
    'sorted_full_data_2021.csv'
]

In [5]:
# Get CoinBase Data
"""
HistoricalData() info:
| ticker | supply the ticker information which you want to return (str). | | granularity | please supply a granularity in seconds (60, 300, 900, 3600, 21600, 86400) (int). | | start_date | a string in the format YYYY-MM-DD-HH-MM (str). | | end_date | a string in the format YYYY-MM-DD-HH-MM (str). Optional, Default: Now | | verbose | printing during extraction. Default: True |
"""
# how many seconds between data points
trade_interval = 60
# Returns data as a dataframe
coinbase_data = HistoricalData(
    'ETH-USD',
    trade_interval,
    start_date='2018-01-01-00-00',
    # end_date='2019-01-6-00-00', # Comment out to use current time as end
    verbose=False
).retrieve_data()

# make time no longer the index and rename it
coinbase_data.reset_index(inplace=True)
coinbase_data.rename(columns = {'time':'timestamp'}, inplace = True)
# Give the index a name for the csv
coinbase_data.index.names = ['index']

# turn timestamp from a datetime into a timestamp, divide by 10**9 to get seconds as our units
coinbase_data['timestamp'] = coinbase_data['timestamp'].apply(lambda x: int(x.value// 10**9))
# Create price data
coinbase_data['decimal_price'] = round(coinbase_data['open'] + coinbase_data['close'] + coinbase_data['high'] + coinbase_data['low'], 4)
# Then fractionalize the number to minimize floating point rounding errors
coinbase_data['fraction_price'] = coinbase_data['decimal_price'].apply(lambda x: frac(x)/4)
# Average out decimal price
coinbase_data['decimal_price'] = coinbase_data['decimal_price']/4
# drop all columns we don't want
coinbase_data = coinbase_data.filter(['index', 'timestamp', 'fraction_price', 'decimal_price'])

print(f'\nSubset of data:\n{coinbase_data.head(5)}')
# Save data
print(f'\nHistorical values found: {len(coinbase_data.index)}')
coinbase_data.to_csv(bs.full_path('CoinBase_ETH_all_price_data'))
print('Done!')


Provisional Start: 2018-01-01T00:00:00
Provisional End: 2018-01-01T05:00:00
Provisional Start: 2018-01-01T05:00:00
Provisional End: 2018-01-01T10:00:00
Provisional Start: 2018-01-01T10:00:00
Provisional End: 2018-01-01T15:00:00
Provisional Start: 2018-01-01T15:00:00
Provisional End: 2018-01-01T20:00:00
Provisional Start: 2018-01-01T20:00:00
Provisional End: 2018-01-02T01:00:00
Provisional Start: 2018-01-02T01:00:00
Provisional End: 2018-01-02T06:00:00
Provisional Start: 2018-01-02T06:00:00
Provisional End: 2018-01-02T11:00:00
Provisional Start: 2018-01-02T11:00:00
Provisional End: 2018-01-02T16:00:00
Provisional Start: 2018-01-02T16:00:00
Provisional End: 2018-01-02T21:00:00
Provisional Start: 2018-01-02T21:00:00
Provisional End: 2018-01-03T02:00:00
Provisional Start: 2018-01-03T02:00:00
Provisional End: 2018-01-03T07:00:00
Provisional Start: 2018-01-03T07:00:00
Provisional End: 2018-01-03T12:00:00
Provisional Start: 2018-01-03T12:00:00
Provisional End: 2018-01-03T17:00:00
Provisional 

KeyboardInterrupt: 

In [4]:
# Get Binance Data

# Initialise the client
# NOTE: You will have to supply your OWN api keys to use this
client = Client(api.get_api_key(), api.get_secret())
# USDT has more historical price data than USDC
trading_pair = 'ETHUSDT'
# trading_pair = 'ETHUSDC'

if trading_pair == 'ETHUSDT':
    ## For ETH USDT
    # Earliest timestamp found: 1502942400.0
    # Human readable format: Wed Aug 16 21:00:00 2017
    # However, early Binance ETH USDT data is not very good so only use 2018+ (aka 1514764800+)
    # Multiply by 1000 to make what we would get from the query
    timestamp = 1514764800*1000
else:
    # get timestamp of earliest date data is available
    timestamp = client._get_earliest_valid_timestamp(trading_pair, AsyncClient.KLINE_INTERVAL_1MINUTE)
    print(f'Earliest timestamp found: {int(timestamp)/1000}')
    print(f'Human readable format: {time.ctime(int(timestamp)/1000)}')
    ## For ETH USDC (not quite as much data as USDT)
    # Earliest timestamp found: 1544844060.0
    # Human readable format: Fri Dec 14 19:21:00 2018

# query_start = '1 Jan, 2019'
# query_end = '6 Jan, 2019'
# # Get data between two specific times
# klines = client.get_historical_klines(trading_pair, AsyncClient.KLINE_INTERVAL_1MINUTE, start_str=query_start, end_str=query_end)

# Get data from the beginning of the Binance data to now
klines = client.get_historical_klines(trading_pair, AsyncClient.KLINE_INTERVAL_1MINUTE, timestamp)

print(f'Historical values found: {len(klines)}')
print(f'Subset of data:\n{klines[0:5]}')

# save data as a file
save_file_name = 'csv_files/Binance_ETH_all_price_data.csv'
with open(save_file_name, 'w') as d:
# with open('csv_files/ETH_all_price_data.csv', 'w') as d:
    d.write('index,timestamp,fraction_price,decimal_price\n')
    # add index column
    ind = 0
    for line in klines:
        fraction_price = frac(float(line[1])+float(line[2])+float(line[3])+float(line[4]))/4
        decimal_price = round(fraction_price, 4)
        # Divide timestamp by 1000 so we can remove three 000s from the timestamp.
        # This makes the timestamp units seconds which matches the other data we have.
        d.write(f'{ind},{int(int(line[0])/1000)},{fraction_price},{decimal_price}\n')
        # keep track of index
        ind += 1

Earliest timestamp found: 1502942400.0
Human readable format: Wed Aug 16 21:00:00 2017
Historical values found: 7201
Subset of data:
[[1546300800000, '131.45000000', '131.54000000', '131.42000000', '131.45000000', '240.20353000', 1546300859999, '31580.25136910', 87, '132.28472000', '17393.36952540', '0'], [1546300860000, '131.44000000', '131.44000000', '131.01000000', '131.22000000', '621.28854000', 1546300919999, '81529.91094440', 140, '262.86286000', '34508.77042620', '0'], [1546300920000, '131.22000000', '131.27000000', '131.19000000', '131.24000000', '64.85972000', 1546300979999, '8511.90702200', 38, '27.04020000', '3548.97590320', '0'], [1546300980000, '131.24000000', '131.30000000', '131.23000000', '131.28000000', '151.86484000', 1546301039999, '19932.27362780', 59, '119.30085000', '15658.01938440', '0'], [1546301040000, '131.27000000', '131.31000000', '131.24000000', '131.27000000', '190.91042000', 1546301099999, '25061.05237230', 80, '42.22663000', '5543.53532960', '0']]


In [ ]:
# Test if csv are sorted
for csv in list_of_csv:
    data = pd.read_csv(bs.full_path(csv))
    sorted_data = data.sort_values(by=['timestamp'])
    print(f'{csv} is sorted?: {data.equals(sorted_data)}')
    # The above resulted in 
    # full_data__6__2018.csv is sorted?: True
    # full_data__6__2019.csv is sorted?: False
    # full_data__6__2020.csv is sorted?: False
    # full_data__6__2021.csv is sorted?: False
    # So we have to sort the data (only do this once)
    
    # Test for nulls
    null_counts = sorted_data[['timestamp', 'Open', 'Close', 'High', 'Low']].isnull().sum()
    null_counts[null_counts > 0].sort_values(ascending=False)
    print('Null values found:')
    print(null_counts)
    # No nulls!

    # Create 'price' column from avg of Open and Close
    # First make the row and round to 4 decimals
    # Divide by 4 after we make the fraction_price
    sorted_data['decimal_price'] = round(sorted_data['Open'] + sorted_data['Close'] + sorted_data['High'] + sorted_data['Low'],
         4)
    # Then fractionalize the number to minimize floating point rounding errors
    sorted_data['fraction_price'] = sorted_data['decimal_price'].apply(lambda x: frac(x)/4)
    # Average out decimal price
    sorted_data['decimal_price'] = sorted_data['decimal_price']/4
    # Give the index a name for the csv
    sorted_data.index.names = ['index']
    # drop all columns we don't want
    sorted_data = sorted_data.filter(['index', 'timestamp', 'fraction_price', 'decimal_price'])
    # Only run this once
    # NOTE: Uncomment this when running for the first time
    sorted_data.to_csv(bs.full_path('sorted_full_data_'+ csv[-8:]))

# Spacing
print('\nSorted check:')
# Show that sorted data is sorted
for csv in sorted_list_of_csv:
    data = pd.read_csv(bs.full_path(csv))
    sorted_data = data.sort_values(by=['timestamp'])
    print(f'{csv} is sorted?: {data.equals(sorted_data)}')



In [ ]:
# Combine ALL data into one big file

# initialize list of all of the files we want to look at
overall_list = [
    'Binance_ETH_all_price_data.csv',
    'CoinBase_ETH_all_price_data.csv'
]
# add the Kaggle data to our list
overall_list = overall_list + sorted_list_of_csv

# initialize empty dataframe to hold all the values
combined_df = pd.DataFrame(columns=['timestamp', 'fraction_price', 'decimal_price'])

# combine all the data into one dataset
for csv in overall_list:
    print(f'Adding {csv}')
    new_data = pd.read_csv(bs.full_path(csv))
    combined_df = idh.combine_datasets(combined_df, new_data)

# Drop the fake index
combined_df = combined_df.drop(columns=['index'])
# Name the real index for the csv
combined_df.index.names = ['index']

# save result to a file
combined_df.to_csv('csv_files/Combined_ETH_all_price_data.csv')
print('Results Saved!\n')

# Check if we have any timestamp gaps in our data
# Ideally we would only see the final timestamp printed
idh.check_missing_timestamp(combined_df)

In [ ]:
# idh.create_price_period('1/1/2018','1/5/2018', 'test')
# idh.create_price_period('1/1/2018','2/1/2018', 'test_month')

# --- Specific price_periods --- 
# Yearly
idh.create_price_period('1/1/2018','1/1/2019', '2018_price_data')
idh.create_price_period('1/1/2019','1/1/2020', '2019_price_data')
idh.create_price_period('1/1/2020','1/1/2021', '2020_price_data')
idh.create_price_period('1/1/2021','1/1/2022', '2021_price_data') 
# Past 4 Years - 2018 through 2021
idh.create_price_period('1/1/2018', '1/1/2022', '2018-2021')
# Past 3 Years - 2019 through 2021
idh.create_price_period('1/1/2019', '1/1/2022', '2019-2021')
# Past 2 Years - 2020 through 2021
idh.create_price_period('1/1/2020', '1/1/2022', '2020-2021')
# Past 1 Year - all of 2021 # Low to high to low to high
idh.create_price_period('1/1/2021', '1/1/2022', '2021')

# High to low 
# - 1515870180 (max of 2018) to end of 2018
idh.create_price_period(1515870180, 1546300740, 'High-Low-1')
# - 1620125000 (before 2021 crash) to 1627000000 (2021 crash low)
idh.create_price_period(1620125000, 1627000000, 'High-Low-2')

# Low to high 
# - start of 2020 to 1620125000 (before 2021 crash)
idh.create_price_period('1/1/2020', 1620125000, 'Low-High-1')
# - 1627000000 (2021 crash low) to end of 2021
idh.create_price_period(1627000000, '1/1/2022', 'Low-High-2')

# Low to high to low
# - all of 2019
idh.create_price_period('1/1/2019', 1577836740, 'Low-High-Low-1')
# - 2021 start to 1627000000 (2021 crash low)
idh.create_price_period('1/1/2021', 1627000000, 'Low-High-Low-2')

# High to low to high
# - 1515870180 (2018) to 1620125000 (before 2021 crash)
idh.create_price_period(1515870180, 1620125000, 'High-Low-High-1')
# - 1620125000 (before 2021 crash) to end of 2021
idh.create_price_period(1620125000, '1/1/2022', 'High-Low-High-2')

In [ ]:
# Recalibrate time variable in a new row to start at 0
# start_time = data['Time'].values[0]
# data['Cal Time'] = data['Time'].apply(lambda x: x - start_time)

# Examples
# Plot data returned
# By default x=index
# data.plot(y='Price', kind='line')
# data.plot(x='Cal Time', y=['Price', 'Plus .3%', 'Minus .3%'], kind='line')
# data.plot(x='Cal Time', y=['Price', 'Plus .3%', 'Minus .3%'], kind='line', xlim=(35000,45000))
# data.plot(x='Cal Time', y=['Price', 'Plus .3%', 'Minus .3%'], kind='line', ylim=(data['Price'].mean()*(1-.005),data['Price'].mean()*(1+.005)))

# testing
data = pd.read_csv(get_test_data_path('test.csv'), index_col='index')
# print(type(data['fraction_price'].values[0]))
# print(dir(data['price'].values[0]))
start_datetime = pd.to_datetime(data['timestamp'].iloc[0], unit='s')
end_datetime = pd.to_datetime(data['timestamp'].iloc[-1], unit='s')
print(f'Start datetime: {start_datetime}')
print(f'End datetime:   {end_datetime}')
data.plot(x='timestamp', y='decimal_price', kind='line')

In [ ]:
# Yearly Price Plots
for csv in sorted_list_of_csv:
    data = pd.read_csv(bs.full_path(csv), index_col='index')
    start_datetime = pd.to_datetime(data['timestamp'].iloc[0], unit='s')
    end_datetime = pd.to_datetime(data['timestamp'].iloc[-1], unit='s')
    print(f'Start datetime: {start_datetime}')
    print(f'End datetime:   {end_datetime}')
    data.plot(x='timestamp', y='decimal_price', title=csv, kind='line')

In [ ]:
csv = 'Combined_ETH_all_price_data.csv'
print(csv)
data = pd.read_csv(bs.full_path(csv), index_col='index')
start_datetime = pd.to_datetime(data['timestamp'].iloc[0], unit='s')
end_datetime = pd.to_datetime(data['timestamp'].iloc[-1], unit='s')
print(f'Start datetime: {start_datetime}')
print(f'End datetime:   {end_datetime}')
data.plot(x='timestamp', y='decimal_price', title=csv, kind='line')